In [194]:
import geopandas
import geopy
import pandas as pd

In [195]:

locator = geopy.Nominatim(user_agent="myGeocoder")
location = locator.geocode("Casa Loma, 1 Austin Terrace, Toronto, , M5R 1X9, Canada")
print(location)
print('Latitude = {}, Longitude = {}'.format(location.latitude, location.longitude))

Casa Loma, 1, Austin Terrace, Tarragon Village, Toronto—St. Paul's, Old Toronto, Toronto, Golden Horseshoe, Ontario, M5R 1X9, Canada
Latitude = 43.6781015, Longitude = -79.409415775


## read and convert locations from a datase4t (CSV)

In [196]:
df = pd.read_csv('address.csv')

# convert address columns to type string to concat them
address_columns = ['Address1','Address2','City','State','Zip','country']

# df['Zip'] = df['Zip'].astype(float).sum().astype(int).astype(str)
df[address_columns] = df[address_columns].fillna('').astype(str)

df['ADDRESS'] = df[address_columns].agg(', '.join, axis=1)

df.head()

,Type,Number,Name,Address1,Address2,City,State,Zip,country,email,ADDRESS
0,manual,1,Emily,10608 Beard Ave,,Austin,Texas,78748,USA,emily@rockman.life,"10608 Beard Ave, , Austin, Texas, 78748, USA"
1,web,2,Granny,Bryson Lane,,Midlothian,Texas,,USA,ebuschang@gmail.com,"Bryson Lane, , Midlothian, Texas, , USA"
2,web,3,Rebecca,,,Los Angeles,CA,,USA,zoe.rockman@gmail.com,", , Los Angeles, CA, , USA"
3,manual,4,Australia,,,Tamworth,,,Australia,NaN,", , Tamworth, , , Australia"
4,web,5,Canada,,,Toronto,ON,M5R 1X9,Canada,NaN,", , Toronto, ON, M5R 1X9, Canada"


## todo
need to add logic to validate addresses - entire process fails with single invalid address right now

In [197]:
from geopy.extra.rate_limiter import RateLimiter

# 1 - conveneint function to delay between geocoding calls
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
# 2- - create location column
df['location'] = df['ADDRESS'].apply(geocode)
# 3 - create longitude, laatitude and altitude from location column (returns tuple)
df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)
# 4 - split point column into latitude, longitude and altitude columns
df[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df['point'].tolist(), index=df.index)

df.head()


,Type,Number,Name,Address1,Address2,City,State,Zip,country,email,ADDRESS,location,point,latitude,longitude,altitude
0,manual,1,Emily,10608 Beard Ave,,Austin,Texas,78748,USA,emily@rockman.life,"10608 Beard Ave, , Austin, Texas, 78748, USA","(10608, Beard Avenue, Tanglewood Forest, Austi...","(30.1690815, -97.83705505214803, 0.0)",30.169082,-97.837055,0.0
1,web,2,Granny,Bryson Lane,,Midlothian,Texas,,USA,ebuschang@gmail.com,"Bryson Lane, , Midlothian, Texas, , USA","(Bryson Lane, Ovilla, Ellis County, Texas, 760...","(32.502031, -96.908033, 0.0)",32.502031,-96.908033,0.0
2,web,3,Rebecca,,,Los Angeles,CA,,USA,zoe.rockman@gmail.com,", , Los Angeles, CA, , USA","(Los Angeles, Los Angeles County, California, ...","(34.0536909, -118.242766, 0.0)",34.053691,-118.242766,0.0
3,manual,4,Australia,,,Tamworth,,,Australia,NaN,", , Tamworth, , , Australia","(Tamworth, Tamworth Regional Council, New Sout...","(-31.0900743, 150.9290159, 0.0)",-31.090074,150.929016,0.0
4,web,5,Canada,,,Toronto,ON,M5R 1X9,Canada,NaN,", , Toronto, ON, M5R 1X9, Canada","(Toronto, Golden Horseshoe, Ontario, Canada, (...","(43.6534817, -79.3839347, 0.0)",43.653482,-79.383935,0.0


## Fun Visualization

In [198]:
import folium
import folium.plugins as plugins
folium_map = folium.Map(location=[30.1690815, -97.83705505214803],
                        zoom_start=1,
                        tiles='CartoDB dark_matter')


plugins.FastMarkerCluster(data=list(zip(df['latitude'].values, df['longitude'].values))).add_to(folium_map)
folium.LayerControl().add_to(folium_map)
folium_map

### Get ISS Times

In [199]:
import requests
import json

def get_ISS_times(latitude, longitude):
    location_url = 'http://api.open-notify.org/iss-pass.json'
    people_count_url = 'http://api.open-notify.org/astros.json'

    parameters = {"lat": latitude, "lon": longitude}

# Make a get request with the parameters.
    response = requests.get(location_url, params=parameters)

    json_data = response.json()

    pass_overs = json_data['response']  #this is a list of dictionaries on risetime and duration
#     times = pass_overs['risetime']
#     duration = pass_overs['duration']
    return pass_overs

In [200]:
# 1 - conveneint function to delay between geocoding calls
## todo: add ratelimiter
# 2- - create location column

df['passovers'] = df.apply(lambda x: get_ISS_times(x['latitude'], x['longitude']), axis=1)

df.head()

,Type,Number,Name,Address1,Address2,City,State,Zip,country,email,ADDRESS,location,point,latitude,longitude,altitude,passovers
0,manual,1,Emily,10608 Beard Ave,,Austin,Texas,78748,USA,emily@rockman.life,"10608 Beard Ave, , Austin, Texas, 78748, USA","(10608, Beard Avenue, Tanglewood Forest, Austi...","(30.1690815, -97.83705505214803, 0.0)",30.169082,-97.837055,0.0,"[{'duration': 129, 'risetime': 1612642029}, {'..."
1,web,2,Granny,Bryson Lane,,Midlothian,Texas,,USA,ebuschang@gmail.com,"Bryson Lane, , Midlothian, Texas, , USA","(Bryson Lane, Ovilla, Ellis County, Texas, 760...","(32.502031, -96.908033, 0.0)",32.502031,-96.908033,0.0,"[{'duration': 323, 'risetime': 1612641956}, {'..."
2,web,3,Rebecca,,,Los Angeles,CA,,USA,zoe.rockman@gmail.com,", , Los Angeles, CA, , USA","(Los Angeles, Los Angeles County, California, ...","(34.0536909, -118.242766, 0.0)",34.053691,-118.242766,0.0,"[{'duration': 581, 'risetime': 1612641555}, {'..."
3,manual,4,Australia,,,Tamworth,,,Australia,NaN,", , Tamworth, , , Australia","(Tamworth, Tamworth Regional Council, New Sout...","(-31.0900743, 150.9290159, 0.0)",-31.090074,150.929016,0.0,"[{'duration': 416, 'risetime': 1612694115}, {'..."
4,web,5,Canada,,,Toronto,ON,M5R 1X9,Canada,NaN,", , Toronto, ON, M5R 1X9, Canada","(Toronto, Golden Horseshoe, Ontario, Canada, (...","(43.6534817, -79.3839347, 0.0)",43.653482,-79.383935,0.0,"[{'duration': 611, 'risetime': 1612642061}, {'..."


In [203]:
import secrets
from datetime import datetime
ts = int("1284101485")

# if you encounter a "year is out of range" error the timestamp
# may be in milliseconds, try `ts /= 1000` in that case
print(datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S'))


def compose_msg(lst, address):
    msg = "The next {0} sightings of the International Space Station at {1} will be on: \n".format(len(lst), address)
    for r in lst:
        date = datetime.utcfromtimestamp(r['risetime']).strftime('%m-%d-%Y')
        time = datetime.utcfromtimestamp(r['risetime']).strftime("%I:%M %p")
        m = str(r['duration']//60)
        s = str(r['duration']%60)
        m = "{} at {} for {} minutes {} seconds. \n".format(date,time,m,s)
        msg +=  m

    
    
    slack_data={
        "text": msg
    }
# can't access secrets file without unnecesary work
#     response = requests.post(
#         secrets.SLACK_URL, data=json.dumps(slack_data),
#         headers={'Content-Type': 'application/json'}
#     )
    print(msg)
        

2010-09-10 06:51:25


In [204]:
df['msg'] = df.apply(lambda x: compose_msg(x['passovers'], x['ADDRESS']), axis=1)



The next 5 sightings of the International Space Station at 10608 Beard Ave, , Austin, Texas, 78748, USA will be on: 
02-06-2021 at 08:07 PM for 2 minutes 9 seconds. 
02-06-2021 at 11:21 PM for 7 minutes 50 seconds. 
02-07-2021 at 12:57 AM for 10 minutes 52 seconds. 
02-07-2021 at 02:35 AM for 8 minutes 10 seconds. 
02-07-2021 at 04:00 PM for 10 minutes 25 seconds. 

The next 5 sightings of the International Space Station at Bryson Lane, , Midlothian, Texas, , USA will be on: 
02-06-2021 at 08:05 PM for 5 minutes 23 seconds. 
02-06-2021 at 09:45 PM for 4 minutes 34 seconds. 
02-06-2021 at 11:21 PM for 8 minutes 57 seconds. 
02-07-2021 at 12:57 AM for 10 minutes 55 seconds. 
02-07-2021 at 02:35 AM for 7 minutes 7 seconds. 

The next 5 sightings of the International Space Station at , , Los Angeles, CA, , USA will be on: 
02-06-2021 at 07:59 PM for 9 minutes 41 seconds. 
02-06-2021 at 09:38 PM for 6 minutes 23 seconds. 
02-06-2021 at 11:17 PM for 6 minutes 23 seconds. 
02-07-2021 at 12:53